In [1]:
import numpy as np
from tqdm import trange
import os
import torch
from torch.utils.data import DataLoader
import torch.nn.functional as F
from torch import nn
from tqdm import tqdm

import mowl
mowl.init_jvm('10g')
from mowl.datasets import PathDataset, Dataset
from mowl.base_models import EmbeddingELModel
from mowl.nn import ELEmModule, ELBoxModule, BoxSquaredELModule
from mowl.utils.data import FastTensorDataLoader

from src.utils import *
from src.elmodule import *

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/usr/local/lib/python3.10/dist-packages/mowl/lib/slf4j-simple-1.7.25.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/usr/local/lib/python3.10/dist-packages/mowl/lib/slf4j-log4j12-1.7.3.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.slf4j.impl.SimpleLoggerFactory]


In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = 'cpu'
print(device)

cpu


In [3]:
#dataset_name = 'family'
#dataset_name = 'pizza'
dataset_name = 'OWL2DL-1'

In [4]:
experiments = get_experimets(dataset_name)

**Models**

**Eval**

**Experiments: Box2EL**

In [5]:
for experiment in experiments: 
    dataset_name = experiment['dataset_name']
    file_name = experiment['file_name']
    format_ = experiment['format_']
    add_noise = experiment['add_noise']                                                                                                               
   
    dataset = PathDataset(ontology_path=f'datasets/bin/{file_name}_train.owl', # we add noise to train
                          testing_path=f'datasets/bin/{dataset_name}_test.owl',
                          validation_path=f'datasets/bin/{dataset_name}_val.owl')
    
    train_ont = preprocess_ontology_el(dataset.ontology)
    test_ont = preprocess_ontology_el(dataset.testing)
    valid_ont = preprocess_ontology_el(dataset.validation)

    dataset = Dataset(train_ont, testing=test_ont, validation=valid_ont)
    
    box2el = ElModel(dataset, 
                     module_name='box2el', 
                     dim=200, 
                     margin=0.1, 
                     batch_size=4096*8, 
                     test_batch_size=32, 
                     epochs=500, 
                     learning_rate=0.001,
                     device=device)
    
    box2el._train()
    
    print(f'{file_name}:')
    print('Membership:')
    metrics_membership = box2el._eval('membership')
    print('Subsumption:')
    metrics_subsumption = box2el._eval('subsumption')
    print()
    save_results(metrics_subsumption, metrics_membership, f'models/results/box2el/{file_name}.txt')

[main] WARN uk.ac.manchester.cs.owl.owlapi.OWLOntologyManagerImpl - Illegal redeclarations of entities: reuse of entity http://www.w3.org/2000/01/rdf-schema#Literal in punning not allowed [Declaration(NamedIndividual(rdfs:Literal)), Declaration(Class(rdfs:Literal))]
[main] WARN uk.ac.manchester.cs.owl.owlapi.OWLOntologyManagerImpl - Illegal redeclarations of entities: reuse of entity http://www.w3.org/1999/02/22-rdf-syntax-ns#XMLLiteral in punning not allowed [Declaration(NamedIndividual(rdf:XMLLiteral)), Declaration(Class(rdf:XMLLiteral)), Declaration(Datatype(rdf:XMLLiteral))]
[main] WARN uk.ac.manchester.cs.owl.owlapi.OWLOntologyManagerImpl - Illegal redeclarations of entities: reuse of entity http://www.w3.org/2000/01/rdf-schema#Literal in punning not allowed [Declaration(NamedIndividual(rdfs:Literal)), Declaration(Class(rdfs:Literal))]
[main] WARN uk.ac.manchester.cs.owl.owlapi.OWLOntologyManagerImpl - Illegal redeclarations of entities: reuse of entity http://www.w3.org/1999/02/2

Main DataLoader: abox
Epoch: 0, Training: EL loss: 3.5919, ABox loss: 0.8372
Epoch: 100, Training: EL loss: 3.0589, ABox loss: 0.2460
Epoch: 200, Training: EL loss: 2.7665, ABox loss: 0.0277
Epoch: 300, Training: EL loss: 2.5593, ABox loss: 0.0268
Epoch: 400, Training: EL loss: 2.4325, ABox loss: 0.0220
OWL2DL-1:
Membership:
MRR: 0.162, Mean Rank: 29.442, Median Rank: 23.000, Hits@1: 0.143, Hits@3: 0.188, Hits@10: 0.263, Hits@100: 0.951
Subsumption:
MRR: 0.332, Mean Rank: 65.812, Median Rank: 8.000, Hits@1: 0.290, Hits@3: 0.337, Hits@10: 0.528, Hits@100: 0.698

Results saved to  models/results/box2el/OWL2DL-1.txt


[main] WARN uk.ac.manchester.cs.owl.owlapi.OWLOntologyManagerImpl - Illegal redeclarations of entities: reuse of entity http://www.w3.org/2000/01/rdf-schema#Literal in punning not allowed [Declaration(NamedIndividual(rdfs:Literal)), Declaration(Class(rdfs:Literal))]
[main] WARN uk.ac.manchester.cs.owl.owlapi.OWLOntologyManagerImpl - Illegal redeclarations of entities: reuse of entity http://www.w3.org/1999/02/22-rdf-syntax-ns#XMLLiteral in punning not allowed [Declaration(NamedIndividual(rdf:XMLLiteral)), Declaration(Class(rdf:XMLLiteral)), Declaration(Datatype(rdf:XMLLiteral))]
[main] WARN uk.ac.manchester.cs.owl.owlapi.OWLOntologyManagerImpl - Illegal redeclarations of entities: reuse of entity http://www.w3.org/2000/01/rdf-schema#Literal in punning not allowed [Declaration(NamedIndividual(rdfs:Literal)), Declaration(Class(rdfs:Literal))]
[main] WARN uk.ac.manchester.cs.owl.owlapi.OWLOntologyManagerImpl - Illegal redeclarations of entities: reuse of entity http://www.w3.org/1999/02/2

Main DataLoader: abox
Epoch: 0, Training: EL loss: 3.5888, ABox loss: 0.8374
Epoch: 100, Training: EL loss: 3.0560, ABox loss: 0.2432
Epoch: 200, Training: EL loss: 2.7624, ABox loss: 0.0281
Epoch: 300, Training: EL loss: 2.5537, ABox loss: 0.0267
Epoch: 400, Training: EL loss: 2.4341, ABox loss: 0.0249
OWL2DL-1_noisy_gnn_0.25:
Membership:
MRR: 0.111, Mean Rank: 29.558, Median Rank: 23.000, Hits@1: 0.070, Hits@3: 0.106, Hits@10: 0.211, Hits@100: 0.951
Subsumption:
MRR: 0.294, Mean Rank: 125.905, Median Rank: 124.500, Hits@1: 0.292, Hits@3: 0.292, Hits@10: 0.302, Hits@100: 0.468

Results saved to  models/results/box2el/OWL2DL-1_noisy_gnn_0.25.txt


[main] WARN uk.ac.manchester.cs.owl.owlapi.OWLOntologyManagerImpl - Illegal redeclarations of entities: reuse of entity http://www.w3.org/2000/01/rdf-schema#Literal in punning not allowed [Declaration(NamedIndividual(rdfs:Literal)), Declaration(Class(rdfs:Literal))]
[main] WARN uk.ac.manchester.cs.owl.owlapi.OWLOntologyManagerImpl - Illegal redeclarations of entities: reuse of entity http://www.w3.org/1999/02/22-rdf-syntax-ns#XMLLiteral in punning not allowed [Declaration(NamedIndividual(rdf:XMLLiteral)), Declaration(Class(rdf:XMLLiteral)), Declaration(Datatype(rdf:XMLLiteral))]
[main] WARN uk.ac.manchester.cs.owl.owlapi.OWLOntologyManagerImpl - Illegal redeclarations of entities: reuse of entity http://www.w3.org/2000/01/rdf-schema#Literal in punning not allowed [Declaration(NamedIndividual(rdfs:Literal)), Declaration(Class(rdfs:Literal))]
[main] WARN uk.ac.manchester.cs.owl.owlapi.OWLOntologyManagerImpl - Illegal redeclarations of entities: reuse of entity http://www.w3.org/1999/02/2

...

Main DataLoader: abox
Epoch: 0, Training: EL loss: 3.5906, ABox loss: 0.8372
Epoch: 100, Training: EL loss: 3.0448, ABox loss: 0.2817
Epoch: 200, Training: EL loss: 2.7572, ABox loss: 0.0454
Epoch: 300, Training: EL loss: 2.5609, ABox loss: 0.0538
Epoch: 400, Training: EL loss: 2.4279, ABox loss: 0.0507
OWL2DL-1_noisy_disjoint_0.5:
Membership:
MRR: 0.059, Mean Rank: 48.640, Median Rank: 31.000, Hits@1: 0.023, Hits@3: 0.046, Hits@10: 0.084, Hits@100: 0.907
Subsumption:
MRR: 0.325, Mean Rank: 74.940, Median Rank: 24.500, Hits@1: 0.306, Hits@3: 0.339, Hits@10: 0.417, Hits@100: 0.688

Results saved to  models/results/box2el/OWL2DL-1_noisy_disjoint_0.5.txt


[main] WARN uk.ac.manchester.cs.owl.owlapi.OWLOntologyManagerImpl - Illegal redeclarations of entities: reuse of entity http://www.w3.org/2000/01/rdf-schema#Literal in punning not allowed [Declaration(NamedIndividual(rdfs:Literal)), Declaration(Class(rdfs:Literal))]
[main] WARN uk.ac.manchester.cs.owl.owlapi.OWLOntologyManagerImpl - Illegal redeclarations of entities: reuse of entity http://www.w3.org/1999/02/22-rdf-syntax-ns#XMLLiteral in punning not allowed [Declaration(NamedIndividual(rdf:XMLLiteral)), Declaration(Class(rdf:XMLLiteral)), Declaration(Datatype(rdf:XMLLiteral))]
[main] WARN uk.ac.manchester.cs.owl.owlapi.OWLOntologyManagerImpl - Illegal redeclarations of entities: reuse of entity http://www.w3.org/2000/01/rdf-schema#Literal in punning not allowed [Declaration(NamedIndividual(rdfs:Literal)), Declaration(Class(rdfs:Literal))]
[main] WARN uk.ac.manchester.cs.owl.owlapi.OWLOntologyManagerImpl - Illegal redeclarations of entities: reuse of entity http://www.w3.org/1999/02/2

Main DataLoader: abox
Epoch: 0, Training: EL loss: 3.5934, ABox loss: 0.8371
Epoch: 100, Training: EL loss: 3.0418, ABox loss: 0.3012
Epoch: 200, Training: EL loss: 2.7534, ABox loss: 0.0449
Epoch: 300, Training: EL loss: 2.5515, ABox loss: 0.0533
Epoch: 400, Training: EL loss: 2.4094, ABox loss: 0.0488
OWL2DL-1_noisy_disjoint_0.75:
Membership:
MRR: 0.249, Mean Rank: 43.247, Median Rank: 29.000, Hits@1: 0.255, Hits@3: 0.332, Hits@10: 0.428, Hits@100: 0.906
Subsumption:
MRR: 0.316, Mean Rank: 73.546, Median Rank: 30.000, Hits@1: 0.292, Hits@3: 0.319, Hits@10: 0.413, Hits@100: 0.661

Results saved to  models/results/box2el/OWL2DL-1_noisy_disjoint_0.75.txt


[main] WARN uk.ac.manchester.cs.owl.owlapi.OWLOntologyManagerImpl - Illegal redeclarations of entities: reuse of entity http://www.w3.org/2000/01/rdf-schema#Literal in punning not allowed [Declaration(NamedIndividual(rdfs:Literal)), Declaration(Class(rdfs:Literal))]
[main] WARN uk.ac.manchester.cs.owl.owlapi.OWLOntologyManagerImpl - Illegal redeclarations of entities: reuse of entity http://www.w3.org/1999/02/22-rdf-syntax-ns#XMLLiteral in punning not allowed [Declaration(NamedIndividual(rdf:XMLLiteral)), Declaration(Class(rdf:XMLLiteral)), Declaration(Datatype(rdf:XMLLiteral))]
[main] WARN uk.ac.manchester.cs.owl.owlapi.OWLOntologyManagerImpl - Illegal redeclarations of entities: reuse of entity http://www.w3.org/2000/01/rdf-schema#Literal in punning not allowed [Declaration(NamedIndividual(rdfs:Literal)), Declaration(Class(rdfs:Literal))]
[main] WARN uk.ac.manchester.cs.owl.owlapi.OWLOntologyManagerImpl - Illegal redeclarations of entities: reuse of entity http://www.w3.org/1999/02/2

Main DataLoader: abox
Epoch: 0, Training: EL loss: 3.5833, ABox loss: 0.8366
Epoch: 100, Training: EL loss: 2.7581, ABox loss: 0.0384
Epoch: 200, Training: EL loss: 2.4214, ABox loss: 0.0513
Epoch: 300, Training: EL loss: 2.2469, ABox loss: 0.0535
Epoch: 400, Training: EL loss: 2.1391, ABox loss: 0.0524
OWL2DL-1_noisy_disjoint_1.0:
Membership:
MRR: 0.216, Mean Rank: 39.832, Median Rank: 14.000, Hits@1: 0.187, Hits@3: 0.285, Hits@10: 0.439, Hits@100: 0.903
Subsumption:
MRR: 0.293, Mean Rank: 138.006, Median Rank: 139.000, Hits@1: 0.286, Hits@3: 0.292, Hits@10: 0.310, Hits@100: 0.440

Results saved to  models/results/box2el/OWL2DL-1_noisy_disjoint_1.0.txt
